In [68]:
import os, csv

In [69]:
class LVQData:
    def __init__(self):
        """
        LVQData Constructor:
            LVQData is a list of input vectors paired with their correct instrument classification.
            LVQNet will iterate over this structure until it converges.
        """
        self.data          = [] # list of tuples 
        self.instrumentMap = {} # map integers with instruments (labels)
        self.instrumentNum = 0  # current integer instrument (to neuron)
    
    def getVectorData(self, index):
        """
        Get the STFT of the Frame (1025 bins) at the specified index
            index must be an valid integer
        """
        return self.data[index][0]
    
    def getVectorLabel(self, index):
        """
        Get the label of the Frame at the specified index
            - index must be an valid integer
        """
        return self.data[index][1]
    
    def getVector(self, index):
        """
        Get the STFT data and label of the Frame at the specified index
            - index must be an valid integer
        """
        return self.data[index][0], self.data[index][1]
    
    def lookupInstrument(self, index):
        """
        Lookup the instument specified by the number of the neuron
            - index must be a valid integer
        """
        return self.instrumentMap[index]
    
    def loadCSV(self, filepath, label):
        """
        Add a STFT datapoint and label to the LVQData list
            - filepath must be valid string from current working directory (os.getcwd() to check)
            - label must be a string describing the instrument of the csv datapoints for entire CSV
        """
        with open(filepath, 'r') as f:
            read = csv.reader(f, delimiter=',')
            read.next()
            
            for row in read:
                data_struct = (arrayParser(row[1]), label) # Tuple with STFT bin list and then the label
                self.data.append(data_struct)
        
            self.instrumentMap[self.instrumentNum] = label
            self.instrumentNum += 1
            
        return self.data

In [70]:
class LVQNeuron:
    def __init__(self, name):
        self.name = name
        self.weights = []
    
    def setWeights(self, weights):
        """
        Import a vector of information (the first entry) for initializing next unique neuron
            - weights is a string from the import of a CSV
        """
        weights = weights.replace('[', '').replace(']', '').split(',')
        
        for weight in weights:
            self.weights.append(float(weight))
        
        
    def __len__(self):
        return len(self.weights)

In [71]:
class LVQNet:
    def __init__(self, inCount, outCount):
        self.inputs   = inCount
        self.outputs  = outCount
        self.alpha    = 0.1
        self.csvCount = 0  # Limit the CSVs input to number of output neurons
        self.neurons  = {} # Numbered index map (to outputs)
              
        for n in range(outCount):
            curr_neuron = LVQNeuron(n)
            self.neurons[n] = curr_neuron
            
    def __len__(self):
        return len(self.neurons)
    
    def getWeights(self, neuronNo):
        """
        Get the current weights of the neuron specified
            - neuronNo must be a valid integer
        """
        return self.neurons[neuronNo].weights
    
    def setWeights(self, neuronNo, newWeights):
        """
        Change the neuron weights with new Weight vector 
        """
        prev = self.neurons[neuronNo]
        
        if len(prev) == len(newWeights):
            prev.weights = newWeights
        
    # STEP 0
    def enterCSV(self, filepath): 
        if self.csvCount >= self.outputs:
            print "Reached limit of neurons"
            return
            
        with open(filepath, 'r') as f:
            read = csv.reader(f, delimiter=',')
            row = read.next()
            curr_neuron = self.neurons[self.csvCount]
            curr_neuron.setWeights(row[1])
            self.csvCount += 1
            
            print "Successfully added neuron from CSV", filepath
            return

    # STEP 3.1
    def edist(self, inputs, weights):
        euclideanDistance = 0
        
        if len(inputs) != len(weights):
            print len(inputs), "is different length than", len(weights)
            return
        
        for i in range(len(inputs)):
            nth = inputs[i] - weights[i]
            nth = nth ** 2
            euclideanDistance += nth
             
        return euclideanDistance ** (0.5)
    
    # STEP 3.2 
    def minDist(self, inputVector):
        scores = [] # Euclidean Distances
        
        for neuron in self.neurons:
            wunit = self.neurons[neuron].weights
            scores.append(self.edist(inputVector, wunit))
        
        minNeuronIndex = scores.index(min(scores))
        return minNeuronIndex
    
    # STEP 4.1
    def calibrate(self, neuronNo, guessNo):
        
        neuron      = dataset.getVectorLabel(neuronNo) 
        inputVector = dataset.getVectorData(neuronNo)
        guess       = dataset.lookupInstrument(guessNo) 
        weights     = self.getWeights(guessNo)          
        
        addfunc = lambda oldWeight, vec: oldWeight + self.alpha * (vec - oldWeight)
        subfunc = lambda oldWeight, vec: oldWeight - self.alpha * (vec - oldWeight)

        if neuron == guess: 
            newWeights = map(addfunc, weights, inputVector) # assign weights as new weights
            self.setWeights(guessNo, newWeights)
        else:
            newWeights = map(subfunc, weights, inputVector) # assign weights as new weights
            self.setWeights(guessNo, newWeights)

    # STEP 5
    def reduceAlpha(self, value):
        self.alpha -= value
        return self.alpha
    
    # Altogether algorithm (TODO use Mean Squared error)
    def run(self, ds):
        # Data is datastructure (LVQData object that has been initialized with all CSVs)
        if (len(ds.data) == 0):
            print "Need to initialize data in order to run the neural net"
        
        else:
            print "Running the algorithm with "
        

In [72]:
def arrayParser(arr):
    # CSV usage: cast string list to python list
    smooth_stage_1 = arr.replace('[', '').replace(']', '').split(',')
    smooth_stage_2 = map(lambda unit: float(unit), smooth_stage_1)
    return smooth_stage_2

In [73]:
### Driver: Outline of the API / Algorithm in use    
if __name__ == '__main__':
    # Create Network with in and out neuron parameters
    koho = LVQNet(1025, 2)
    
    # Enter data (1-1 CSV to Output Neurons) Initializes the neurons with first onset
    koho.enterCSV('./data/snareFrames.csv')
    koho.enterCSV('./data/kickDrumFrames.csv')
    
    # Instantiate LVQ Training Data Structure and load rest of CSVs with labels
    dataset = LVQData()
    dataset.loadCSV('./data/snareFrames.csv', 'snare')
    dataset.loadCSV('./data/kickDrumFrames.csv', 'kick-drum')
    
    # (TODO) Put the koho minDists in loop / logic
    guess = koho.minDist(dataset.getVectorData(2))  # using a specific sample frame (snare)
    guess2 = koho.minDist(dataset.getVectorData(8)) # using another sample frame (kick)
    
    # (TODO) Test calibrations
    newWeights  = koho.calibrate(2, guess)
    newWeights2 = koho.calibrate(8, guess2)

Successfully added neuron from CSV ./data/snareFrames.csv
Successfully added neuron from CSV ./data/kickDrumFrames.csv
